In [66]:
import pickle

system = 'ibmq_mumbai'
N = 10
T = 1
filename = 'circuits/{}_{}_{}.pickle'.format(system,N,T)

with open(filename,'rb') as f:
    circuits = pickle.load(f)

qc = circuits[0]

In [67]:
rx_calibrations = []
rzx_calibrations = []

for a,b in qc.calibrations['rx'].items():
    rx_calibrations.append({"qubit" : [a[0][0]], "calibration": b, "backend" : system})

for a,b in qc.calibrations['rzx'].items():
    rzx_calibrations.append({'qubit':[a[0][0],a[0][1]],"calibration":b, "backend" : system})

In [68]:
rx_calibrations

[{'qubit': [0],
  'calibration': ScheduleBlock(Play(Drag(duration=160, sigma=40, beta=-0.30079591624152224, amp=0.017325763338610325, angle=0), DriveChannel(0)), name="rx(0.333)", transform=AlignLeft()),
  'backend': 'ibmq_mumbai'},
 {'qubit': [12],
  'calibration': ScheduleBlock(Play(Drag(duration=160, sigma=40, beta=-1.010916880598726, amp=0.011200636048367447, angle=0), DriveChannel(12)), name="rx(0.333)", transform=AlignLeft()),
  'backend': 'ibmq_mumbai'},
 {'qubit': [1],
  'calibration': ScheduleBlock(Play(Drag(duration=160, sigma=40, beta=-1.0747699978090437, amp=0.018747212463171153, angle=0), DriveChannel(1)), name="rx(0.333)", transform=AlignLeft()),
  'backend': 'ibmq_mumbai'},
 {'qubit': [2],
  'calibration': ScheduleBlock(Play(Drag(duration=160, sigma=40, beta=0.7771137782506223, amp=0.017381941418124207, angle=0), DriveChannel(2)), name="rx(0.333)", transform=AlignLeft()),
  'backend': 'ibmq_mumbai'},
 {'qubit': [3],
  'calibration': ScheduleBlock(Play(Drag(duration=160, 

In [69]:
rzx_calibrations

[{'qubit': [14, 11],
  'calibration': ScheduleBlock(ScheduleBlock(ScheduleBlock(Play(GaussianSquare(duration=256, sigma=64, width=0, amp=0.21986057016508842, angle=0.4082948506000275), ControlChannel(29)), name="rescale_cr_inst", transform=AlignLeft()), ScheduleBlock(Play(GaussianSquare(duration=256, sigma=64, width=0, amp=0.022013024482376076, angle=-0.0058549652746592955), DriveChannel(11)), name="rescale_cr_inst", transform=AlignLeft()), name="block14581", transform=AlignLeft()), ScheduleBlock(Play(Drag(duration=160, sigma=40, beta=-0.8150518151804168, amp=0.1833364757739792, angle=0.0, name='Xp_d14'), DriveChannel(14), name='Xp_d14'), name="x", transform=AlignLeft()), ScheduleBlock(ScheduleBlock(Play(GaussianSquare(duration=256, sigma=64, width=0, amp=0.21986057016508836, angle=-2.7332978029897657), ControlChannel(29)), name="rescale_cr_inst", transform=AlignLeft()), ScheduleBlock(Play(GaussianSquare(duration=256, sigma=64, width=0, amp=0.022013024482376076, angle=3.135737688315134

In [70]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.transpiler import InstructionProperties
from qiskit_ibm_provider import IBMProvider
from qiskit_experiments.library import InterleavedRB
from qiskit import QuantumCircuit

In [71]:
provider = IBMProvider()

In [72]:
def conduct_rx_interleaved_rb_experiment(config):

    #Circuit to only pass as a gate
    gate_circuit = QuantumCircuit(1, name='custom_rx')
    gate_circuit.rx(np.pi/2,0)
    custom_rx = gate_circuit.to_gate()

    #Adding calibration
    backend = provider.get_backend(config['backend'])
    backend.target.add_instruction(custom_rx, {(config['qubit'][0],): InstructionProperties(calibration=config['calibration'])})
    
    #Experiment parameters
    lengths = np.arange(1, 200, 30)
    num_samples = 10
    seed = 1010
    qubits = config['qubit'] # make sure it is the qubit added in the backend target

    int_exp = InterleavedRB(
        custom_rx, qubits, lengths, num_samples=num_samples, seed=seed, backend=backend)
    
    int_expdata = int_exp.run(backend)
    config['job_id'] = int_expdata.job_ids[0]

    return config

def conduct_rzx_interleaved_rb_experiment(config):

    #Circuit to only pass as a gate
    gate_circuit = QuantumCircuit(2, name='custom_rzx')
    gate_circuit.rzx(np.pi/2,0,1)
    custom_rzx = gate_circuit.to_gate()

    #Adding calibration
    backend = provider.get_backend(config['backend'])
    backend.target.add_instruction(custom_rzx, {(config['qubit'][0],config['qubit'][1]): InstructionProperties(calibration=config['calibration'])})

    #Experiment parameters
    lengths = np.arange(1, 200, 30)
    num_samples = 10
    seed = 1010
    qubits = config['qubit'] # make sure it is the qubit added in the backend target

    int_exp = InterleavedRB(
        custom_rzx, qubits, lengths, num_samples=num_samples, seed=seed, backend=backend)
    
    int_expdata = int_exp.run(backend)
    config['job_id'] = int_expdata.job_ids[0]

    return config

In [73]:
job_id = conduct_rx_interleaved_rb_experiment(rx_calibrations[0])

In [74]:
job_id

{'qubit': [0],
 'calibration': ScheduleBlock(Play(Drag(duration=160, sigma=40, beta=-0.30079591624152224, amp=0.017325763338610325, angle=0), DriveChannel(0)), name="rx(0.333)", transform=AlignLeft()),
 'backend': 'ibmq_mumbai',
 'job_id': 'cq0psd528rp0008a5590'}

In [76]:
job_id = conduct_rzx_interleaved_rb_experiment(rzx_calibrations[0])

In [77]:
job_id

{'qubit': [14, 11],
 'calibration': ScheduleBlock(ScheduleBlock(ScheduleBlock(Play(GaussianSquare(duration=256, sigma=64, width=0, amp=0.21986057016508842, angle=0.4082948506000275), ControlChannel(29)), name="rescale_cr_inst", transform=AlignLeft()), ScheduleBlock(Play(GaussianSquare(duration=256, sigma=64, width=0, amp=0.022013024482376076, angle=-0.0058549652746592955), DriveChannel(11)), name="rescale_cr_inst", transform=AlignLeft()), name="block14581", transform=AlignLeft()), ScheduleBlock(Play(Drag(duration=160, sigma=40, beta=-0.8150518151804168, amp=0.1833364757739792, angle=0.0, name='Xp_d14'), DriveChannel(14), name='Xp_d14'), name="x", transform=AlignLeft()), ScheduleBlock(ScheduleBlock(Play(GaussianSquare(duration=256, sigma=64, width=0, amp=0.21986057016508836, angle=-2.7332978029897657), ControlChannel(29)), name="rescale_cr_inst", transform=AlignLeft()), ScheduleBlock(Play(GaussianSquare(duration=256, sigma=64, width=0, amp=0.022013024482376076, angle=3.135737688315134),

In [ ]:
rx_filename = 'error_rates/{}_{}_rx_config.pkl'.format(system,T)
rzx_filename = 'error_rates/{}_{}_rzx_config.pkl'.format(system,T)

with open(rx_filename,'wb+') as f:
    pickle.dump(rx_calibrations,f)

with open(rzx_filename,'wb+') as f:
    pickle.dump(rzx_calibrations,f)

Adding data from job failed [Job ID: cq0pc10hbttg008j7tgg]
jobs and analysis raised exceptions [Experiment ID: c178179c-8531-43c8-8ba8-9eedd71eae24]:Traceback (most recent call last):

  File "/Users/amey/anaconda3/envs/simuq/lib/python3.10/site-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(

  File "/Users/amey/anaconda3/envs/simuq/lib/python3.10/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):

  File "/Users/amey/anaconda3/envs/simuq/lib/python3.10/socket.py", line 955, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):

socket.gaierror: [Errno 8] nodename nor servname provided, or not known


The above exception was the direct cause of the following exception:


Traceback (most recent call last):

  File "/Users/amey/anaconda3/envs/simuq/lib/python3.10/site-packages/urllib3/connectionpool.py", li